In [1]:
import pandas as pd
import geopandas as gpd
import os

In [2]:
# Open the data on the wave fetchs

fetch_length = gpd.read_file('SHP/Fetch_length.shp').drop(columns=['Distance', 'OBJECTID', 'X', 'Y', 'geometry'])    # Open the fetch length dataset

fetch_length['Bearing'] = fetch_length['Bearing'].replace({180: 'S', 135: 'SE', 157.5: 'SSE', 202.5: 'SSW', 225: 'SW'})   # Rename wind directions from degrees to rhumbs

fetch_length['Shape_Leng'] = fetch_length['Shape_Leng']/1000    # Convert distances from meters to kilometers

In [3]:
# Calculate the wind wave effect for one transect for one year

def wwe_year_mi(TransectID, meteo_df):
    
    fetch_length1 = fetch_length[(fetch_length['TransectID'] == TransectID)]
    
    grad_cols = ['WSW', 'SW', 'SSW', 'S','SSE', 'SE']    # Create a list with the wind rhumbs
    grad_index = ['5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45', '45-50', '>50']    # Create a list with the wind gradations
    Gradations = pd.DataFrame(columns = grad_cols, index = grad_index)    # Construct a dataframe to calculate a frequency of winds of different gradations of different rhumbs
    Av_wind_speed = Gradations.copy()    # Construct a dataframe to calculate an average speed of winds of different gradations of different rhumbs
   
    # Iterate the rhumbs and the gradations and calculate the frequency and the average speed for them
    for rhumb in grad_cols:
        for speed in grad_index[:-1]:
            lb = int(speed.split('-')[0])
            rb = int(speed.split('-')[1])
            Gradations[rhumb][speed] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed'] > lb) & (meteo_df['wind_speed'] <= rb)].count()
            Av_wind_speed[rhumb][speed] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed'] > lb) & (meteo_df['wind_speed'] <= rb)].mean()

        Gradations[rhumb]['>50'] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed']>50)].count()
        Av_wind_speed[rhumb]['>50'] = meteo_df['wind_speed'][(meteo_df['wind_dir']==rhumb) & (meteo_df['wind_speed']>50)].mean()
 
    v3 = (Av_wind_speed.fillna(0))**3    # Raise to the 3rd power the wind speed for not null values
   
    wwe = Gradations.copy()    # Create a dataframe for wind-wave effect values
    
    # Iterate the rhumbs and calculate the wind-wave effect for them
    for i, rhumb in wwe.items():
        if rhumb.name in fetch_length1['Bearing'].unique():
            for j in range(0, len(rhumb)):
                wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_length1[(fetch_length1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
        else:
            wwe[rhumb.name] = 0        
    
    return wwe.values.sum()

In [4]:
# Open the files with weather station data for different years

folder = 'CSV/years'
filelist = os.listdir(folder)

In [5]:
# Create a dataframe for any year dataset

def df_making(file, folder):
    meteo_df = pd.read_csv(os.path.join(folder, file))
    meteo_df = meteo_df.drop(meteo_df[meteo_df['wind_speed'] == '---'].index)
    meteo_df['wind_speed'] = meteo_df['wind_speed'].astype(float)
    return meteo_df

In [6]:
# Construct a dataframe for all the years

wwe_all = pd.DataFrame(index = fetch_length['TransectID'].unique(), columns=[filelist])

In [7]:
wwe_all

,2010.csv,2011.csv,2012.csv,2013-1.csv,2013-2.csv,2014.csv,2015-1.csv,2015-2.csv,2016.csv,2017-1.csv,2017-2.csv,2018.csv,2019_2020.csv,2020.csv
216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Iterate files with different years, calculate a WWE for every transect every year and fill in the resulted dataframe

for file in filelist:
    meteo_df = df_making(file, folder)
    wwe_year = pd.DataFrame(index = fetch_length['TransectID'].unique(), columns=[file])
    for TransectID in fetch_length['TransectID'].unique():
        transect_wwe = wwe_year_mi(TransectID, meteo_df)
        wwe_year[file][TransectID] = transect_wwe
    wwe_all[file] = wwe_year[file]  

C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


...

C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600
C:\Users\anovikova\AppData\Local\Temp\ipykernel_24752\1003566886.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wwe[rhumb.name][rhumb.index[j]] = 0.000003 * fetch_id1[(fetch_id1['Bearing'] == rhumb.name)]['Shape_Leng'].iloc[0] * v3[rhumb.name][rhumb.index[j]] * Gradations[rhumb.name][rhumb.index[j]] * 600


In [12]:
wwe_all

,2010.csv,2011.csv,2012.csv,2013-1.csv,2013-2.csv,2014.csv,2015-1.csv,2015-2.csv,2016.csv,2017-1.csv,2017-2.csv,2018.csv,2019.csv,2020.csv
1,12052020.576849,22097278.006079,34051244.524008,37520029.401645,14550909.862465,24294861.859824,23938943.039309,7088355.025202,44002943.72385,29613120.012352,9353077.390211,45559483.133858,84077154.618983,14884568.639488
2,12052145.245599,22097256.641574,34051281.966676,37519726.69877,14550182.28696,24294816.916215,23940325.068813,7091127.208456,44009868.749249,29620397.025894,9354634.474805,45567020.019795,84087093.614891,14887700.923751
3,12052643.848464,22097957.262855,34052542.911525,37520780.771404,14550240.346863,24295660.758489,23941612.712784,7092864.353108,44014679.245913,29625087.466626,9355692.078586,45572215.262415,84094485.076912,14889658.074256
4,12053073.760491,22098643.91065,34053595.133917,37521747.097452,14550614.403111,24296434.958457,23942289.856688,7093111.202871,44015843.187957,29626035.304769,9355975.969201,45573463.2509,84096862.7083,14890090.401278
5,12053803.368431,22099595.317339,34055357.355298,37522933.565715,14550861.084848,24297649.283879,23942499.060299,7092828.596359,44015124.610368,29625285.997992,9355799.080487,45572428.954923,84096126.295803,14889584.853029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,11394464.175659,19840098.058432,31220584.97857,33123587.829776,7695589.135394,21251401.070511,29684265.564207,19910662.089552,76551950.025003,62888173.449229,16045476.719883,78575140.415095,126020066.817201,29799287.835174
466,11393121.826957,19838045.899758,31217508.387152,33120992.723455,7694440.648197,21249066.22389,29682477.359984,19910204.840684,76549566.095932,62886097.85648,16044792.649289,78572447.742037,126014307.819532,29798372.224801
467,11392062.507344,19836684.723617,31214950.730201,33119248.612055,7694716.92523,21247459.296553,29681361.091076,19909695.865728,76547552.949952,62884550.868394,16044498.219578,78571175.775132,126011490.358704,29797758.657259
468,11391291.483778,19835576.083199,31212707.743348,33117327.54347,7694623.136839,21246068.112626,29680511.426944,19909077.485306,76545332.947595,62882953.777544,16044185.828047,78569376.426851,126008484.074786,29797215.810155


In [10]:
# Export the resulted dataframe to csv

#wwe_all.to_csv('CSV/wwe_all_year_transects.csv')